In [2]:
# Scraping no Site QuintoAndar
# Camila Junqueira
# Gabriel Nasser
# Rafaela Alexandre

import requests
from bs4 import BeautifulSoup
import json
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from pprint import pprint

In [3]:
def getDadosCEP(cep):
		url = (f'http://www.viacep.com.br/ws/{cep}/json')
		
		req = requests.get(url)
		if req.status_code == 200:
			dados_json = json.loads(req.text)
			return dados_json
		else:
			print('Erro ao buscar CEP')

In [4]:
meu_cep = "04078012"
dados = getDadosCEP(meu_cep)
bairro = dados['bairro']
dados

{'cep': '04078-012',
 'logradouro': 'Avenida Divino Salvador',
 'complemento': 'de 501/502 a 799/800',
 'unidade': '',
 'bairro': 'Planalto Paulista',
 'localidade': 'São Paulo',
 'uf': 'SP',
 'estado': 'São Paulo',
 'regiao': 'Sudeste',
 'ibge': '3550308',
 'gia': '1004',
 'ddd': '11',
 'siafi': '7107'}

In [5]:
bairro = bairro.replace(" ", "-").lower()
bairro

'planalto-paulista'

In [6]:
qa = "https://www.quintoandar.com.br/comprar/imovel/{bairro}-sao-paulo-sp-brasil?pagina=10"

In [113]:
def scraper(bairro):
    driver = webdriver.Chrome()

    driver.get('https://www.quintoandar.com.br/comprar/imovel/'+bairro+'-sao-paulo-sp-brasil')
    time.sleep(5)

    for i in range(9):
        try:
            ver_mais_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Ver mais')]")
            ver_mais_button.click()  # Clica no botão
            time.sleep(3)  # Aguarde o conteúdo carregar após o clique
        except:
            pass

    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    soup = soup.find('html')
    soup = soup.prettify()
    return soup

In [11]:
req = requests.get("https://www.quintoandar.com.br/comprar/imovel/planalto-paulista-sao-paulo-sp-brasil?pagina=10")
soup = BeautifulSoup(req.content, 'html.parser')
soup = soup.find('html')
casas = soup.find_all(attrs={"data-testid": "house-card-container"})
len(casas)


12

: 

In [100]:
moema = scraper(bairro)
pprint(moema)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=128.0.6613.138)
Stacktrace:
	GetHandleVerifier [0x00007FF6E0D49412+29090]
	(No symbol) [0x00007FF6E0CBE239]
	(No symbol) [0x00007FF6E0B7B1DA]
	(No symbol) [0x00007FF6E0B4FAF5]
	(No symbol) [0x00007FF6E0BFE2C7]
	(No symbol) [0x00007FF6E0C15EB1]
	(No symbol) [0x00007FF6E0BF6493]
	(No symbol) [0x00007FF6E0BC09D1]
	(No symbol) [0x00007FF6E0BC1B31]
	GetHandleVerifier [0x00007FF6E106871D+3302573]
	GetHandleVerifier [0x00007FF6E10B4243+3612627]
	GetHandleVerifier [0x00007FF6E10AA417+3572135]
	GetHandleVerifier [0x00007FF6E0E05EB6+801862]
	(No symbol) [0x00007FF6E0CC945F]
	(No symbol) [0x00007FF6E0CC4FB4]
	(No symbol) [0x00007FF6E0CC5140]
	(No symbol) [0x00007FF6E0CB461F]
	BaseThreadInitThunk [0x00007FFC483C257D+29]
	RtlUserThreadStart [0x00007FFC48C2AF28+40]


In [14]:
soup = BeautifulSoup(moema, 'html.parser')
casas = soup.find_all(attrs={"data-testid": "house-card-container"})
casas[0]

<div data-testid="house-card-container">
<a class="StyledLink_styledLink__P_6FN" href="/imovel/894622729/comprar/casa-4-quartos-planalto-paulista-sao-paulo?from_route=%22search_results%22&amp;house_tags=isSaleGoodPrice&amp;house_tags=newAd&amp;search_id=%2250758009-21d8-4330-8633-ca4fdf929933%22&amp;search_rank=%7B%22sortMode%22%3A%22relevance%22%2C%22searchMode%22%3A%22list%22%2C%22resultsOrigin%22%3A%22search%22%2C%22rank%22%3A0%2C%22personalization%22%3Atrue%7D" target="_blank" title="Casa à venda com 4 quartos, 300m² em Planalto Paulista, São Paulo">
<div aria-label="Casa . Ótimo preço, Anúncio novo. Planalto Paulista, São Paulo, Alameda dos Quinimuras. 300 metros quadrados, 4 quartos, 2 vagas de garagem.  R$ 0 Condo. + IPTU, R$ 750.000 ." class="qHIgfb CsxKHh Cozy__FindHouseCard-Container" role="group" tabindex="0">
<div aria-live="polite" class="Cozy__CardBlock-Container _2pfuCF znH6Fs">
<div class="Cozy__CardRow-Container oVdjIf" role="complementary" style="--card-row-top: 0; --

In [82]:
rua = casas[0].find("div", class_="Cozy__FindHouseCard-Container").text
rua = rua.split("São Paulo")[0].split('\n\n')[-1].lstrip()
rua = rua.split(",")[0]
rua

'Alameda dos Quinimuras'

In [93]:
def find_rua_imovel(imovel):
    rua = imovel.find("div", class_="Cozy__FindHouseCard-Container").text
    rua = rua.split("São Paulo")[0].split('\n\n')[-1].lstrip()
    rua = rua.split(",")[0]
    return rua

In [34]:
valor = casas[0].find("div", class_="Cozy__FindHouseCard-Container").text
valor = valor.split("R$")[1]
valor = re.sub(r'\D', '', valor)
valor

'750000'

In [33]:
img_tag = casas[0].find('img', class_='ProgressiveImage_image__1QoR0')
alt_text = img_tag['alt']
alt_text = alt_text.split("m²")[0].split(" ")
alt_text[-1]


'300'

In [39]:
def find_preco_imovel(imovel):
    valor = imovel.find("div", class_="Cozy__FindHouseCard-Container").text
    valor = valor.split("R$")[1]
    return re.sub(r'\D', '', valor)

In [40]:
find_preco_imovel(casas[0])

'750000'

In [41]:
def find_m2_imovel(imovel):
    img_tag = imovel.find('img', class_='ProgressiveImage_image__1QoR0')
    alt_text = img_tag['alt']
    alt_text = alt_text.split("m²")[0].split(" ")
    return alt_text[-1]
    

In [25]:
find_m2_imovel(casas[0])

'300'

In [44]:
for i in range(0, 30):
    print("Preco: " + find_preco_imovel(casas[i]))
    print("m2: " + find_m2_imovel(casas[i]))
    print("-"*10)

Preco: 750000
m2: 300
----------
Preco: 630000
m2: 144
----------
Preco: 387000
m2: 50
----------
Preco: 800000
m2: 131
----------
Preco: 660000
m2: 90
----------
Preco: 799000
m2: 120
----------
Preco: 720000
m2: 160
----------
Preco: 610000
m2: 95
----------
Preco: 780000
m2: 140
----------
Preco: 460000
m2: 96
----------
Preco: 450000
m2: 69
----------
Preco: 800000
m2: 120
----------
Preco: 599000
m2: 128
----------
Preco: 600000
m2: 86
----------
Preco: 335000
m2: 27
----------
Preco: 399000
m2: 60
----------
Preco: 450000
m2: 69
----------
Preco: 800000
m2: 150
----------
Preco: 460000
m2: 96
----------
Preco: 799000
m2: 120
----------
Preco: 535000
m2: 120
----------
Preco: 870000
m2: 220
----------
Preco: 200000
m2: 18
----------
Preco: 2498000
m2: 350
----------
Preco: 1240000
m2: 130
----------
Preco: 426000
m2: 58
----------
Preco: 549000
m2: 92
----------
Preco: 630000
m2: 144
----------
Preco: 1450000
m2: 160
----------
Preco: 1540000
m2: 430
----------


In [45]:
import sqlite3

In [86]:
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

In [87]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    cep TEXT NOT NULL,
    rua TEXT NOT NULL,
    bairro TEXT NOT NULL,
    cidade TEXT NOT NULL,
    estado TEXT NOT NULL,
    regiao TEXT NOT NULL,
    preco FLOAT NOT NULL,
    m2 FLOAT NOT NULL,
    preco_m2 FLOAT NOT NULL
)
''')

In [89]:
def add_data(cep, rua, bairro, cidade, estado, regiao, preco, m2, preco_m2):
    cursor.execute('''
    INSERT INTO data (cep, rua, bairro, cidade, estado, regiao, preco, m2, preco_m2) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (cep, rua, bairro, cidade, estado, regiao, preco, m2, preco_m2))
    conn.commit()

In [49]:
def remove_data(id):
    cursor.execute('''
    DELETE FROM data WHERE id = ?
    ''', (id,))
    conn.commit()

In [50]:
def get_id(cep):
    cursor.execute('''
    SELECT id FROM data WHERE cep = ?
    ''', (cep,))
    data = cursor.fetchall()
    return data

In [97]:
def extract_data(cep):
    dados = getDadosCEP(cep)
    bairro = dados['bairro']
    cidade = dados['localidade']
    estado = dados['uf']
    regiao = dados['regiao']
    bairro = bairro.replace(" ", "-").lower()
    soup = scraper(bairro)
    soup = BeautifulSoup(soup, 'html.parser')
    casas = soup.find_all(attrs={"data-testid": "house-card-container"})
    for casa in casas:
        preco = find_preco_imovel(casa)
        m2 = find_m2_imovel(casa)
        rua = find_rua_imovel(casa)
        preco_m2 = float(preco) / float(m2)
        
        # Check if the data already exists
        cursor.execute('''
        SELECT id FROM data 
        WHERE cep = ? AND bairro = ? AND preco = ? AND m2 = ?
        ''', (cep, bairro, preco, m2))
        
        if cursor.fetchone() is None:
            # Data doesn't exist, so insert it
            add_data(cep, rua, bairro, cidade, estado, regiao, preco, m2, preco_m2)

In [101]:
extract_data("04078012")

In [52]:
def delete_all():
    cursor.execute('''
    DELETE FROM data
    ''')
    conn.commit()

In [95]:
# delete_all()
# Limpar banco de dados para teste 

In [66]:
def precos_medios():
    cursor.execute('''
    SELECT bairro, AVG(preco_m2) AS preco_medio_m2
    FROM data
    GROUP BY bairro
    ''')
    return cursor.fetchall()

[('planalto-paulista', 7625.514712091567)]

In [105]:
extract_data("04119010")

In [111]:
precos_medios()

[('moema', 15195.022755120612),
 ('planalto-paulista', 7702.220294383596),
 ('vila-mariana', 1678897.5215833574)]

In [112]:
for bairro, preco_medio_m2 in precos_medios():
    print(f"Bairro: {bairro} - Preço médio por m²: R${preco_medio_m2:.2f}")

Bairro: moema - Preço médio por m²: R$15195.02
Bairro: planalto-paulista - Preço médio por m²: R$7702.22
Bairro: vila-mariana - Preço médio por m²: R$1678897.52
